# Extract Poses into NPY Files (Mimic PoseRAC's pre_test.py)

In [31]:
import numpy as np
import pandas as pd
import glob
import os
import sys
import deeplabcut

In [32]:
# System alterations

# Set the print options to avoid truncation
np.set_printoptions(threshold=sys.maxsize) # Default threshold is 1000

In [33]:
# Note that parameters of this project can be seen at: *openfield-Pranav-2018-10-30/config.yaml*
PATH_CONFIG_FILE = os.path.join(os.getcwd(), 'examples', 'CowBytes-Single-Sadat-2024-06-30', 'config.yaml')
deeplabcut.load_demo_data(PATH_CONFIG_FILE)

This is not an official demo dataset.
Loaded, now creating training data...
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [89]:
# Define test poses destination directory and video file paths [EDIT HERE]
TEST_POSES_DEST_DIR = os.path.join(os.getcwd(), 'examples', 'BiteCountA_pose', 'test_poses')
ROOT_VIDEO_DIR = os.path.join(os.getcwd(), 'examples', 'BiteCountA_pose', 'video')
TRAIN_VIDEO_DIR = os.path.join(ROOT_VIDEO_DIR, 'train')
VALID_VIDEO_DIR = os.path.join(ROOT_VIDEO_DIR, 'valid')
TEST_VIDEO_DIR = os.path.join(ROOT_VIDEO_DIR, 'test')

In [35]:
# Create test_poses directory
os.makedirs(TEST_POSES_DEST_DIR, exist_ok=True)

In [ ]:
# Extract poses into .npy files
video_dir = [TRAIN_VIDEO_DIR, VALID_VIDEO_DIR, TEST_VIDEO_DIR]

for dir in video_dir:
    deeplabcut.analyze_videos(PATH_CONFIG_FILE,dir,save_as_csv=True)

In [ ]:
# Load all generated .csv files
csv_files = glob.glob(os.path.join(ROOT_VIDEO_DIR, '**', '*.csv'), recursive=True)
print("Number of generated CSV Files:", len(csv_files))
print(csv_files[:5])

In [ ]:
for csv_file in csv_files:
    # Process CSV file contents
    df = pd.read_csv(csv_file, header=None)
    df = df.drop([0, 1, 2]).reset_index(drop=True)
    df.drop(columns=df.columns[0], axis=1, inplace=True)
    datapoints = df.values.astype(np.float32)
    datapoints[:, 2::3] = 0
    # Save NPY file with datapoints
    dest_file = os.path.join(TEST_POSES_DEST_DIR, os.path.basename(csv_file.split('DLC')[0]))
    np.save(dest_file, datapoints)
    print("Created file: ", dest_file)

In [ ]:
# Remove redundant files in video directories
files = glob.glob(os.path.join(ROOT_VIDEO_DIR, '**', '*.*'), recursive=True)

# Delete non-JPG files
for file in files:
    if not file.endswith('.mp4'):
        try:
            os.remove(file)
            print(f"Deleted: {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")